# 1️⃣ Contexte

## Contexte 

- [**Sirene**]{.orange} est le répertoire des entreprises françaises
- Un [**code APE**]{.orange} est attribué à chaque entreprise
- Début 2023:
  - [**Refonte**]{.blue2} du répertoire Sirene
  - Mise en place d'un [**guichet unique**]{.blue2} pour les formalités d'entreprises
  - [**Forte baisse de performance**]{.blue2} de l'outil de codification historique
  - Equipes Sirene déjà [**surchargées**]{.blue2} avant la refonte

- [**Conséquence**]{.orange} : besoin d'un nouvel outil... en innovant

## Données

- Prédiction du code APE à partir des descriptifs d'activités
- [$\approx$ **10 millions**]{.orange} d'obs. issues de Sirene 3 couvrant la période 2014-2022
- [**Données labellisées**]{.orange} par Sicore ou par un gestionnaire
- Structure [**hiérarchique**]{.orange} de la nomenclature

## La nomenclature hiérarchique de l'APE

| Niveau | Code | Libellé | Taille |
|-------|------|----------------------|-|
| Section     | H     |Transports et entreposage|21|
| Division    | 52    |Entreposage et services auxiliaires des transports|88|
| Groupe      | 522   |Services auxiliaires des transports|272|
| Classe      | 5224  |Manutention|615|
| **Sous-classe** | [**5224A**]{.red2} |**Manutention portuaire**|[**732**]{.orange}|


## Données utilisées {.scrollable}

::: {.panel-tabset}

### Données 

  - Une observation consiste en : 
  - [Description textuelle]{.blue2} de l'activité - [text]{.green2}
    - [**Code APE vrai**]{.blue2} labelisé par le moteur de règles –- [APE_SICORE]{.green2} (732 modalités)
    - un [**descriptif textuel**]{.red2} de l'activité -- [LIB_SICORE]{.green2}
    - la [**nature de l’activité**]{.blue2} -- [NAT]{.green2} (23 modalités)
    - le [**type de la liasse**]{.blue2} -- [TYP]{.green2} (15 modalités)
    - le [**type d’évènement**]{.blue2} -- [EVT]{.green2} (24 modalités)
    - la [**surface ($m^2$)**]{.blue2}-- [SUR]{.green2} (4 modalités)

- [**Prétraitements**]{.orange} standards :
  - Passage en minuscules
  - Suppression de la ponctuation
  - Suppression des nombres
  - Suppression des *stop words*
  - Racinisation (*stemming*)
  - ...


### Brutes

```{ojs}
viewof table_data = Inputs.table(transpose(data_raw), {
    rows: 22
})
```

### Pré-traitée

```{ojs}
viewof table_data_prepro = Inputs.table(transpose(data_prepro), {
    rows: 22
})
```

:::


In [ ]:
import sys
import os
import pandas as pd
import s3fs
import pyarrow.parquet as pq
import unidecode
import string
import numpy as np
from nltk.corpus import stopwords as ntlk_stopwords
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stopwords = tuple(ntlk_stopwords.words("french")) + tuple(string.ascii_lowercase)
stemmer = SnowballStemmer(language="french")

def clean_lib(df: pd.DataFrame, text_feature: str) -> pd.DataFrame:
    """
    Cleans a text feature for pd.DataFrame `df` at index idx.

    Args:
        df (pd.DataFrame): DataFrame.
        text_feature (str): Name of the text feature.
        method (str): The method when the function is used (training or
        evaluation)

    Returns:
        df (pd.DataFrame): DataFrame.
    """
    # pylint: disable=C0301

    # On harmonise l'encodage (principalement suppression accents)
    df[text_feature] = df[text_feature].map(unidecode.unidecode)

    # On passe tout en minuscule
    df[text_feature] = df[text_feature].str.lower()

    # define replacement patterns
    replacements = {
        # Libellé vide de sens fournit par Christine (DOIT ETRE FAIT EN AMONT DU MODELE EN JAVA)
        r"\bidem\b|\bvoir ci dessous\b|\[vide\]|\bundefined\b|\bpas d objet\b|\(voir ci dessus\)|\(voir extrait siege social\\/etablissement principal\)|\bcf activite principale\b|\bcf activite principale et objet\b|\bcf activites de l entreprise\b|\bcf activites principales de l entreprise\b|\bcf actvites principales\b|\bcf k bis\b|\bcf le principales activites de l  entreprise\b|\bcf le sprincipale activites de l  entreprise\b|\bcf le sprincipales activites de l  entreprise\b|\bcf les activites principales de l  entreprise\b|\bcf les ppales activites de l  entreprise\b|\bcf les ppales activites de la ste\b|\bcf les principale activites de l  entreprise\b|\bcf les principales activites\b|\bcf les principales activites de l  entreprise\b|\bcf les principales activites de l  entreprises\b|\bcf les principales activites ppales de l  entreprise\b|\bcf les principales activtes de l  entreprise\b|\bcf les principales acttivites de l  entreprise\b|\bcf les prinipales activites de l  entreprise\b|\bcf lesprincipales activites de l  entreprise\b|\bcf objet\b|\bcf obs\b|\bcf principales activite de l  entreprise\b|\bcf principales activites de l  entreprise\b|cf rubrique \"principales activites de l entreprise\" idem|cf rubrique n2 ci dessus \(743b\)|\bcf supra\b|\bcf ci  dessus\b|\bcommerce de detail, idem case 2\b|\bextension a: voir ci dessus\b|\bid\b|\bid principales activites\b|\bid principales activites de l  entreprise\b|\bidem ci dessus\b|idem \( voir principales activites\)|\bidem  dessus\b|\bidem 1ere page\b|\bidem a principales activites de l  entreprise\b|\bidem activiet eprincipale\b|\bidem activite\b|\bidem activite 1ere page\b|\bidem activite ci  dessus\b|\bidem activite de l  entreprise\b|\bidem activite enoncee ci  dessus\b|\bidem activite entreprise\b|\bidem activite generales\b|\bidem activite premiere page\b|\bidem activite principale\b|\bidem activite princippale\b|\bidem activite prinicpale\b|\bidem activite sur 1ere page\b|\bidem activites ci dessus\b|\bidem activites declarees au siege et principal\b|\bidem activites enoncees ci dessus\b|\bidem activites entreprise\b|\bidem activites principales\b|\bidem activites principales de l entreprise\b|\bidem activites siege\b|\bidem activte principale\b|\bidem activtie 1ere page\b|\bidem au siege\b|\bidem au siege social\b|\bidem aux principales actiivtes\b|\bidem aux principales activites\b|\bidem case 13\b|\bidem ci dessous\b|\bidem ci dessus enoncee\b|\bidem cidessus\b|\bidem objet\b|\bidem premiere page\b|\bidem pricincipales activites de l entreprise\b|\bidem pricipales activites\b|\bidem principale activite\b|\bidem principales activite de l entreprise\b|\bidem principales activite de l entreprises\b|\bidem principales activite l entreprise\b|\bidem principales activites\b|\bidem principales activites citees ci dessus\b|\bidem principales activites de l entreprises\b|idem principales activites de l entreprise\(objet\)|\bidem principales activites et objet social\b|\bidem principales activitse de l entreprise\b|\bidem que celle decrite plus haut\b|\bidem que ci dessus\b|\bidem que l activite decrite plus haut\b|\bidem que les activites principales\b|\bidem que les activites principales ci dessus\b|\bidem que les activitges principales\b|\bidem que les principales activites\b|\bidem que les principales activites de l entreprise\b|\bidem que pour le siege\b|\bidem rubrique principales activites de l entreprise\b|\bidem siege\b|idem siege \+ voir observation|\bidem siege et ets principal\b|\bidem siege social\b|idem siege, \(\+ articles americains\)|\bidem societe\b|\bidem voir activite principale\b|\bidem voir ci dessus\b|\bidentique a l objet social indique en case 2 de l imprime m2\b|\bidm ci dessus\b|\bnon indiquee\b|\bnon precise\b|\bnon precisee\b|\bnon precisees\b|\bvoir 1ere page\b|\bvoir activite ci dessus\b|\bvoir activite principale\b|\bvoir activite principale ci dessus\b|\bvoir activites principales\b|\bvoir cidessus\b|\bvoir idem ci dessus\b|\bvoir objet social\b|\bvoir page 1\b|\bvoir page precedente\b|\bvoir plus haut\b|\bvoir princiale activite\b|\bvoir princiales activites\b|\bvoir princiapales activites\b|\bvoir princiaples activites\b|\bvoir principale activite\b|\bvoir principales activites\b|\bvoir principales activites de l entreprise\b|\bvoir principales actvites\b|\bvoir principalesactivites\b|\bvoir principles activites\b|\bvoir rubrique principales activites de l entreprise\b|\bvoir sur la 1ere page\b|\bvoir dessus\b|voir: \"principales activite de l entreprise\"|voir: \"principales activites de l entreprises\"|voir: \"principales activites de l entrprise\"|voir: \"principales activites en entreprise\"|\bconforme au kbis\b|\bsans changement\b|\bsans activite\b|\bsans acitivite\b|\bactivite inchangee\b|\bactivites inchangees\b|\bsiege social\b|\ba definir\b|\ba preciser\b|\bci dessus\b|\bci desus\b|\bci desssus\b|\bvoir activit principale\b|\bidem extrait kbis\b|\bn a plus a etre mentionne sur l extrait decret\b|\bcf statuts\b|\bactivite principale case\b|\bactivites principales case\b|\bactivite principale\b|\bactivites principales\b|\bvoir case\b|\baucun changement\b|\bsans modification\b|\bactivite non modifiee\b|\bactivite identique\b|\bpas de changement\b|\bcode\b|\bape\b|\bnaf\b|\binchangee\b|\binchnagee\b|\bkbis\b|\bk bis\b|\binchangees\b|\bnp\b|\binchange\b|\bnc\b|\bxx\b|\bxxx\b|\binconnue\b|\binconnu\b|\bvoir\b|\bannexe\b|\bmo\b|\biem\b|\binchanges\b|\bactivite demeure\b|\bactivite inchangée\b|\bcase precedente\b|\bidem cadre precedent\b|\bactivite demeure\b|\bactivite inchangée\b|\bnon renseignee\b|\bneant\b|\bnon renseigne\b": " ",
        # supprime hyphen pour les mots comme e-commerce
        r"e-": "e",
        # accole le e pour les mots comme e-commerce
        r"\be\s": " e",
        # On supprime toutes les ponctuations
        r"[^\w\s]": " ",
        # On supprime certains mots sans sens (DOIT ETRE FAIT DANS LE PREPROCESSING EN JAVA)
        r"\bcode\b|\bcadre\b|\bape\b|\bape[a-z]{1}\b|\bnaf\b|\binchangee\b|\binchnagee\b|\bkbis\b|\bk bis\b|\binchangees\b|\bnp\b|\binchange\b|\bnc\b|\bidem\b|\bxx\b|\bxxx\b|\baa\b|\baaa\b|\bidem cadre precedent\b|\bidem case\b|\binchanges\b|\bmo\b|\biem\b|\bci dessus\b|\bet\b": "",
        # On supprime tous les chiffres
        r"[\d+]": " ",
    }

    # apply replacements to text_feature column
    for pattern, replacement in replacements.items():
        df[text_feature] = df[text_feature].str.replace(pattern, replacement, regex=True)

    # On supprime les mots d'une seule lettre
    df[text_feature] = df[text_feature].apply(
        lambda x: " ".join([w for w in x.split() if len(w) > 1])
    )

    # define replacement patterns
    replacements = {
        # On supprime les espaces multiples
        r"\s\s+": " ",
        # On strip les libellés
        r"^\s+|\s+$": "",
        # On remplace les empty string par des NaN
        r"^\s*$": np.nan,
    }

    # apply replacements to text_feature column
    for pattern, replacement in replacements.items():
        df[text_feature] = df[text_feature].replace(pattern, replacement, regex=True)

    # On tokenize tous les libellés
    libs_token = [lib.split() for lib in df[text_feature].to_list()]

    # On supprime les mots duppliqué dans un même libellé
    libs_token = [
        sorted(set(libs_token[i]), key=libs_token[i].index) for i in range(len(libs_token))
    ]

    # Pour chaque libellé on supprime les stopword et on racinise les mots
    df[text_feature] = [
        " ".join(
            [stemmer.stem(word) for word in libs_token[i] if word not in stopwords]
        )
        for i in range(len(libs_token))
    ]

    return df

In [ ]:
DATA_PATH="projet-ape/data/data_sirene3.parquet"
TEXT_FEATURE="LIB_SICORE"

kwargs = {
    "key": os.environ["AWS_ACCESS_KEY_ID"],
    "secret": os.environ["AWS_SECRET_ACCESS_KEY"]
}

if "AWS_SESSION_TOKEN" in os.environ:
    kwargs["token"] = os.environ["AWS_SESSION_TOKEN"]

fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": f"""https://{os.environ["AWS_S3_ENDPOINT"]}"""}, **kwargs
)
df = pq.ParquetDataset(DATA_PATH, filesystem=fs).read_pandas().to_pandas()
df = df.sample(frac=0.001)

df = df[["APE_SICORE", "LIB_SICORE", "AUTO", "NAT_SICORE", "EVT_SICORE", "SURF"]]
dff = df.copy()
df_prepro = clean_lib(df, TEXT_FEATURE)

ojs_define(data_raw = dff, data_prepro = df_prepro)

# 2️⃣ Méthodologie

## Modèle fastText {background-image="../img/diag-fasttext.png" background-size="95%" background-position="50% 75%"} 

- [**fastText**]{.orange}: Modèle "sac de mots" très [**simple**]{.blue2} et [**rapide**]{.blue2} (`C++`).


## Prise en compte des variables catégorielles

- [**Concaténation**]{.orange} du libellé avec les noms et valeurs des variables annexes :

Libellé | NAT | TYP | EVT | SUR | 
--------|-----|-----|-----|-----|
Cours de musique | NaN | X | 01P |NaN

<center>
&#129155;
</center>

"Cours de musique [NAT]{.green2}\_[NaN]{.blue2} [TYP]{.green2}\_[X]{.blue2} [EVT]{.green2}\_[01P]{.blue2} [SUR]{.green2}\_[NaN]{.blue2}"

- [**Méthode imparfaite**]{.orange} : 3-grams "AT_" ou "T_0" utilisés.




# 3️⃣ Situation actuelle

## Le paysage administratif

![](../img/flux-formalites.png){fig-align="center"}

## Construction d'un indice de confiance {.scrollable}

![](../img/IC.png){fig-align="center"}

- [**Objectif**]{.orange} : [**discriminer**]{.blue2} les *mauvaises* des *bonnes* prédictions.
- [**Indice de confiance retenu**]{.orange} : différence entre les deux probabilités les plus élevées.

## Mise en production actuelle

- 2 issues possibles :
    - codification automatique 🚀
    - reprise gestionnaire 🔍

![](../img/orga-post-prod-fr.png){fig-align="center"}

## Codification du flux de liasses

![](../img/table-passage-codification.png){fig-align="center"}

## Reprise gestionnaire

![](../img/mode-reprise-gestionnaire.png){fig-align="center"}

## Codification automatique sur l'IHM

![](../img/automatic-coding-HMI.png){fig-align="center"}

## Reprise gestionnaire sur l'IHM

![](../img/reprise_gestionnaire_manque_contexte.png){fig-align="center"}







# 4️⃣ Vers une approche MLOps

## Situation actuelle inefficace

- [**Infrastructure**]{.orange} divergente : 
    - Modèle entrainé sur la plateforme [**SSP Cloud**]{.blue2} 😍
    - Module de codification [**développé en Java**]{.blue2} au sein d'une architecture monolithique 😫
    - [**Duplication de code**]{.blue2} et risque d'erreurs 

## Situation actuelle inefficace

- Problèmes [**organisationnels**]{.orange} : 
    - [**Méconnaissance des problèmes métier**]{.blue2} côté Lab pour la maintenance
    - [**Contrôle de version**]{.blue2} et [**transmission**]{.blue2} des modèles non centralisés
    - Organisation [**inefficace**]{.blue2} entre les parties prenantes

## Situation actuelle inefficace

- [**Spécificités du ML**]{.orange} non prises en compte :
  - Pas de [**surveillance**]{.blue2} du modèle en temps réel
  - Pas de données [**golden standard**]{.blue2}
  - Pas de [**ré-entrainement**]{.blue2} régulier

- [**Conséquence**]{.orange} : Besoin d'adopter les [**bonnes pratiques**]{.blue2} d'un point de vue technique (MLOps) et organisationel

![](../img/mlops-cycle.png){fig-align="center"}

## Notre implémentation MLOps 🚀

- Architecture en microservices qui tournent sur un [**cluster Kubernetes**]{.orange} :
  - Contrôle de version des expérimentations et modèles : [**MLflow**]{.blue2}
  - Mise à dispostion du modèle via API: [**FastAPI**]{.blue2}
  - Automatisation avec [**ArgoCD**]{.blue2}
  - Dashboard de surveillance : [**Quarto**]{.blue2} and [**DuckDB**]{.blue2}
  - Contrôle qualité: annotations avec [**Label Studio**]{.blue2}

## MLflow au coeur de notre projet

![](../img/mlflow-model-registry.png){fig-align="center"}


##  Exposer un modèle via une API

- Modèle mis à disposition via une [**API**]{.orange} conteneurisée :
  - [**Simple**]{.blue2} pour les utilisateurs

```{ojs}
viewof activite = Inputs.text( 
  {label: '', value: 'Institut National de la Santé et de la Recherche Médicale (INSERM)', width: 800}
)

urlApe = `https://codification-ape-test.lab.sspcloud.fr/predict?nb_echos_max=4&prob_min=0&text_feature=${activite}&type_liasse=C&event=01P`

d3.json(urlApe).then( res => {
  var IC, results;
  ( {IC, ...results} = res )

  IC = parseFloat(IC)

  const rows = Object.values(results).map( obj => {
    return `
    <tr>
      <td>${obj.code} | ${obj.libelle}</td>
      <td>${obj.probabilite.toFixed(3)}</td>
    </tr>
  `
  }).join('')
  
  return html`
  <table>
    <caption>
      Indice de confiance : ${IC.toFixed(3)}
    </caption>
    <tr>
      <th style="text-align:center;">Libellé (NA2008)</th>
      <th>Probabilité</th>
    </tr>
      ${rows}
  </table>`
})
```


```{ojs}
import { debounce } from "@mbostock/debouncing-input" 
```


##  Exposer un modèle via une API

- Modèle mis à disposition via une [**API**]{.orange} conteneurisée :
  - [**Simple**]{.blue2} pour les utilisateurs
  - [**Format de requête standard**]{.blue2}
  - Passage à l'[**échelle**]{.blue2} optimisé
  - [**Modulaire**]{.blue2} et [**portable**]{.blue2}
- Déploiement continu avec [**Argo CD**]{.orange}


## Exposer un modèle via une API

![](../img/api-datalab.png){fig-align="center"}

## Monitoring 

- [**Surveiller**]{.orange} un modèle ML en production est nécessaire :
  - Pour détecter des [**distribution drifts**]{.blue2} dans les données d'entrées
  - Pour s'assurer qu'il a un [**comportement stable**]{.blue2}
  - Pour savoir [**quand ré-entraîner**]{.blue2} le modèle
- Idéallement, on voudrait connaître la [**performance en temps réel**]{.orange} mais compliqué.. et coûteux !
- Surveillance [**technique**]{.orange} aussi importante : [**latence**]{.orange}, [**memoire**]{.orange}, [**usage disque**]{.orange}, etc.


## Monitoring 

![](../img/monitoring-datalab){fig-align="center"}

## Contrôle qualité

- Données de test [**collectées et annotées périodiquement**]{.orange}
- Annotation réalisée avec [**Label Studio**]{.orange}
- [**Métriques de performance**]{.orange} calculées sur le jeu de test
- Performance affichée dans un [**dashboard de surveillance**]{.orange}
- [**Ré-entraînement**]{.orange} déclenché lorsque certaines métriques passent un certain seuil (pas encore fait)

## Contrôle qualité

![](../img/annotation-datalab.png){fig-align="center"}

## Dashboard

[Tableau de bord](https://dashboard-ape.lab.sspcloud.fr/){preview-link="true"}
